In [1]:
import numpy as np
import math
import typing
import time
import scipy
from scipy.stats import binom, multinomial

"""
このファイルは、以下の論文に基づいてMaxCutのQAOAプロキシアルゴリズムを実装しています:
https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.6.023171
"""

# 論文におけるP(c')の計算
def prob_cost_paper(cost: int, num_constraints: int, prob_edge: float = 0.5) -> float:
    return binom.pmf(cost, num_constraints, prob_edge)


# 論文におけるN(c')の計算
def number_with_cost_paper_proxy(cost: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    scale = 1 << num_qubits  # 2のnum_qubits乗
    return prob_cost_paper(cost, num_constraints, prob_edge) * scale


# 論文におけるP(b, c'-b, c-b | d)の計算
def prob_common_at_distance_paper(num_constraints: int, common_constraints: int, cost_1: int, cost_2: int, distance: int) -> float:
    # 同じ制約がある確率
    prob_same = (math.comb(num_constraints - distance, 2) + math.comb(distance, 2)) / math.comb(num_constraints, 2)
    prob_neither = prob_same / 2
    prob_both = prob_neither
    prob_one = (1 - prob_neither - prob_both) / 2
    return multinomial.pmf(
        [common_constraints, cost_1 - common_constraints, cost_2 - common_constraints, num_constraints + common_constraints - (cost_1 + cost_2)],
        num_constraints,
        [prob_both, prob_one, prob_one, prob_neither],
    )


# 論文におけるN(c'; d, c)の計算
def number_of_costs_at_distance_paper_proxy(cost_1: int, cost_2: int, distance: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    sum = 0
    # 共通の制約数の範囲を設定し、その範囲内での確率を計算
    for common_constraints in range(max(0, cost_1 + cost_2 - num_constraints), min(cost_1, cost_2) + 1):
        sum += prob_common_at_distance_paper(num_constraints, common_constraints, cost_1, cost_2, distance)

    p_cost = prob_cost_paper(cost_1, num_constraints, prob_edge)
    return (math.comb(num_qubits, distance) / p_cost) * sum


# 論文のアルゴリズム1のループ内での合計計算
def compute_amplitude_sum_paper(prev_amplitudes: np.ndarray, gamma: float, beta: float, cost_1: int, num_constraints: int, num_qubits: int) -> complex:
    sum = 0
    # cost_2とdistanceの範囲でのループ
    for cost_2 in range(num_constraints + 1):
        for distance in range(num_qubits + 1):
            # βとγのファクターを計算
            beta_factor = (np.cos(beta) ** (num_qubits - distance)) * ((-1j * np.sin(beta)) ** distance)
            gamma_factor = np.exp(-1j * gamma * cost_2)
            num_costs_at_distance = number_of_costs_at_distance_paper_proxy(cost_1, cost_2, distance, num_constraints, num_qubits)
            sum += beta_factor * gamma_factor * prev_amplitudes[cost_2] * num_costs_at_distance
    return sum


# TODO: 期待値プロキシの最適化ではなく、高コスト振幅の最適化を行う場合の検討
# 論文のアルゴリズム1
# num_constraintsはエッジの数、num_qubitsは頂点の数
def QAOA_paper_proxy(p: int, gamma: np.ndarray, beta: np.ndarray, num_constraints: int, num_qubits: int):
    num_costs = num_constraints + 1
    amplitude_proxies = np.zeros([p + 1, num_costs], dtype=complex)
    init_amplitude = np.sqrt(1 / (1 << num_qubits))  # 初期振幅の計算
    for i in range(num_costs):
        amplitude_proxies[0][i] = init_amplitude

    # 各深さでの振幅を計算
    for current_depth in range(1, p + 1):
        for cost_1 in range(num_costs):
            amplitude_proxies[current_depth][cost_1] = compute_amplitude_sum_paper(
                amplitude_proxies[current_depth - 1], gamma[current_depth - 1], beta[current_depth - 1], cost_1, num_constraints, num_qubits
            )

    expected_proxy = 0
    # 最終深さでの期待値を計算
    for cost in range(num_costs):
        expected_proxy += number_with_cost_paper_proxy(cost, num_constraints, num_qubits) * (abs(amplitude_proxies[p][cost]) ** 2) * cost

    return amplitude_proxies, expected_proxy


# 逆問題の目的関数を生成
def inverse_paper_proxy_objective_function(num_constraints: int, num_qubits: int, p: int, expectations: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        _, expectation = QAOA_paper_proxy(p, gamma, beta, num_constraints, num_qubits)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        return -expectation

    return inverse_objective


# QAOAの実行と最適化
def QAOA_paper_proxy_run(
    num_constraints: int,
    num_qubits: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    expectations: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])  # 初期のγとβを結合

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_paper_proxy_objective_function(num_constraints, num_qubits, p, expectations),
        init_freq,
        args=(),
        method=optimizer_method,
        options=optimizer_options,
    )
    end_time = time.time()

    # 経過時間を相対的に計算する関数
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))

    gamma, beta = result.x[:p], result.x[p:]
    _, expectation = QAOA_paper_proxy(p, gamma, beta, num_constraints, num_qubits)

    return {
        "gamma": gamma,
        "beta": beta,
        "expectation": expectation,
        "runtime": end_time - start_time,  # 経過時間（秒単位）
        "num_QAOA_calls": result.nfev,  # QAOAの呼び出し回数
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message,
    }
